<a href="https://colab.research.google.com/github/Ary8888/SaanviAryanaAppProject/blob/main/pink_tax_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary packages
!pip install streamlit scikit-learn pandas numpy pyngrok

# Import libraries
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle
import streamlit as st
from pyngrok import ngrok
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00


In [ ]:
# colab_model_training.py

import pandas as pd
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from google.colab import files
uploaded = files.upload()

# Load CSV
df = pd.read_csv("final dataset1.csv")  # Replace with your filename
df = df.dropna()
df['Price'] = df['Price'].replace('[\$,]', '', regex=True).astype(float)

print(df.columns)

# One-hot encode manually
# One-hot encode using available columns
# One-hot encode using available columns
categorical_cols = ['Brand', 'Item Type', 'Gender']  # or whatever you choose
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()  # <- this part fixes it

# Store encoded feature names
encoded_feature_names = encoder.get_feature_names_out(categorical_cols)

# Create full feature set
X = pd.DataFrame(encoded_features, columns=encoded_feature_names)
y = df['Price']


# Split data (optional)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Train model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Save model and encoder
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

with open("encoder.pkl", "wb") as f:
    pickle.dump(encoder, f)

print("✅ Model and encoder saved!")



In [ ]:
!mkdir -p streamlit_app

In [ ]:
# streamlit_app.py

import streamlit as st
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

# Load model and encoder
with open("model.pkl", "rb") as f:
    model = pickle.load(f)

with open("encoder.pkl", "rb") as f:
    encoder = pickle.load(f)

# Load original dataset
@st.cache_data
def load_data():
    df = pd.read_csv("final dataset1.csv")  # Match the original CSV
    df['Price'] = df['Price'].replace('[\$,]', '', regex=True).astype(float)
    df = df.dropna()
    return df

df = load_data()

# Sidebar input
st.title("💸 Pink Tax Price Predictor (Offline Model)")
brand = st.sidebar.selectbox("Brand", df['Brand'].unique())
item_type = st.sidebar.selectbox("Item Type", df['Item Type'].unique())
gender = st.sidebar.selectbox("Gender", df['Gender'].unique())

# Input DataFrame
input_data = pd.DataFrame({
    'Brand': [brand],
    'Item Type': [item_type],
    'Gender': [gender]
})

# One-hot encode input
encoded_input = encoder.transform(input_data).toarray()
encoded_input_df = pd.DataFrame(encoded_input, columns=encoder.get_feature_names_out())


# Predict
predicted_price = model.predict(encoded_input_df)[0]

st.subheader("Predicted Price (Neutral Model)")
st.write(f"💰 Estimated Price: **${predicted_price:.2f}**")

# Gender-based visuals
st.subheader("📊 Gender-Based Price Discrepancies")

avg_prices = df.groupby('Gender')['Price'].mean().reset_index()
fig, ax = plt.subplots()
sns.barplot(data=avg_prices, x='Gender', y='Price', palette='pastel', ax=ax)
ax.set_title("Average Price by Gender")
st.pyplot(fig)

st.subheader("📉 Price Differences by Category")
gender_pivot = df.pivot_table(values='Price', index='Item Type', columns='Gender', aggfunc='mean')
gender_pivot['Discrepancy ($F - $M)'] = gender_pivot['F'] - gender_pivot['M']

st.dataframe(gender_pivot.sort_values('Discrepancy ($F - $M)', ascending=False).style.format("{:.2f}"))


# New Section

In [ ]:
import os

!pip install -q streamlit
!npm install localtunnel
!wget -q -O - ipv4.icanhazip.com # USE THIS OUTPUT (ex., '34.133.84.111') as the Tunnel Password if you are asked on the next page
!streamlit run streamlit_app/app.py & npx localtunnel --port 8501

In [ ]:
streamlit run streamlit_app.py


In [ ]:
import streamlit as st
import pandas as pd
import pytesseract
from flask import Flask, request, jsonify
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


def clean_data(df, category):
    # Your data cleaning code here
    return df

# Load and preprocess datasets
def load_data(filepath):
    df = pd.read_csv(filepath)
    df = df.dropna()
    df['Category'] = category
    df['Price'] = df['Price'].astype(float)
    return df

# Clean datasets
deodorant_df = clean_data(deodorant_df, 'Deodorant')
razor_df = clean_data(razor_df, 'Razor')

# Combine both datasets
data = pd.concat([deodorant_df, razor_df], ignore_index=True)

# Identify gendered products
data['Gender'] = data['Product Name'].apply(lambda x: 'Men' if 'Men' in x else ('Women' if 'Women' in x else 'Unisex'))

# Calculate Average Prices by Gender and Category
avg_prices = data.groupby(['Category', 'Gender'])['Price'].mean().reset_index()

# Pink Tax Analysis
def pink_tax_analysis(df, category):
    men_price = df[(df['Category'] == category) & (df['Gender'] == 'Men')]['Price'].mean()
    women_price = df[(df['Category'] == category) & (df['Gender'] == 'Women')]['Price'].mean()
    tax_percentage = ((women_price - men_price) / men_price) * 100 if men_price else 0
    return men_price, women_price, tax_percentage

# Compute Pink Tax for each category
pink_tax_data = {}
for category in data['Category'].unique():
    men_price, women_price, tax_percentage = pink_tax_analysis(data, category)
    pink_tax_data[category] = {'Men': men_price, 'Women': women_price, 'Pink Tax %': tax_percentage}

# Convert to DataFrame
pink_tax_df = pd.DataFrame.from_dict(pink_tax_data, orient='index')
print(pink_tax_df)

# Visualization
plt.figure(figsize=(10,5))
sns.barplot(x=pink_tax_df.index, y=pink_tax_df['Pink Tax %'], palette='coolwarm')
plt.xlabel("Category")
plt.ylabel("Pink Tax Percentage")
plt.title("Gender-Based Price Disparity (Pink Tax)")
plt.show()

# Store Locator Placeholder
def find_cheaper_alternatives(product_name, category):
    category_df = data[(data['Category'] == category) & (data['Gender'] == 'Unisex')]
    if not category_df.empty:
        return category_df.sort_values('Price').iloc[0]
    return "No cheaper alternative found"

# Example Usage
cheaper_alternative = find_cheaper_alternatives("Gillette Razor Women", "Razor")
print("Cheaper Alternative:", cheaper_alternative)
